In [ ]:
import json
import random
import logging
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support

!pip install sense2vec==1.0.3
!pip uninstall spacy
!pip install spacy==2.3.7

import spacy
from spacy.gold import GoldParse
from spacy.scorer import Scorer
from sklearn.metrics import accuracy_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.4 MB 18.6 MB/s 
     |████████████████████████████████| 184 kB 72.8 MB/s 
     |████████████████████████████████| 1.0 MB 63.3 MB/s 
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.4
    Uninstalling srsly-2.4.4:
      Successfully uninstalled srsly-2.4.4
  Attempting uninstall: catalogue
    Found existing installation: catalogue 2.0.8
    Uninstalling catalogue-2.0.8:
      Successfully uninstalled catalogue-2.0.8
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.0
    Uninstalling thinc-8.1.0:
      Successfully uninstalled thinc-8.1.0
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.1
    Uninstalling spacy-3.4.1:
      Successfully uninstalled spacy-3.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavi

In [ ]:
print( spacy.__version__ )

2.3.7


In [ ]:
import json

In [ ]:
# data['annotations']['points']['text']

'family'

In [ ]:
dict1 = {
    "label": "Person",
    "points": [{
        "start": 100,
        "end": 105,
        "text": "family"
    }]
}

dict2 = {
    "label": "Person",
    "points": [{
        "start": 110,
        "end": 116,
        "text": "friends"
    }]
}

data = \
{
    "content": "glkjhdfsglkjsh",
    "annotation": [dict1]
}

data['annotation'].append( dict2 )

# data_string = str( data )
# json_object = json.loads( data_string )

json_object = json.dumps( data )
# json_object = json.dumps( json.loads( json_object ), indent=4 )

with open( "/content/ejemplo.json", "a" ) as outfile:
  outfile.write( json_object )
  outfile.write( "\n" )

In [ ]:
with open( "/content/ejemplo.json", "a" ) as outfile:
  outfile.write( json_object )
  outfile.write( "\n" )

In [ ]:
def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
    try:
        training_data = []
        lines=[]
        with open(dataturks_JSON_FilePath, 'r') as f:
            lines = f.readlines()

        for line in lines:
            data = json.loads(line)
            text = data['content']
            entities = []
            for annotation in data['annotation']:
                #only a single point in text annotation.
                point = annotation['points'][0]
                labels = annotation['label']
                # handle both list of labels or a single label.
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    #dataturks indices are both inclusive [start, end] but spacy is not [start, end)
                    entities.append((point['start'], point['end'] + 1 ,label))


            training_data.append((text, {"entities" : entities}))

        return training_data
    except Exception as e:
        logging.exception("Unable to process " + dataturks_JSON_FilePath + "\n" + "error = " + str(e))
        return None

# sauce: https://github.com/DataTurks-Engg/Entity-Recognition-In-Resumes-SpaCy/blob/master/train.py

In [ ]:
convert_doccano_to_spacy( "/content/admin.jsonl" )

[('"Social media and news black out?\nI\'m at work and I just had a troubling realization. Now I\'m sure a lot of people know what\'s happened this past weekend in america. I won\'t go into it here but obviously it\'s horrific. I\'ve been feeling a little shaky all day, the urge to escape my worries and anxiety swirling in my mind.\n\nI should clarify I\'ve long been a drinker to try to combat my social anxiety and constant worry about literally anything and everything at any given moment. But these last four days have made me realize I cannot go back to drinking, never again. It\'s just a crutch and I need to get over that.\n\nSo I\'m here at work trying to keep my mind busy and everyone is talking about the events of the weekend everywhere I go. I can\'t escape it.\n\nMy question is, is it necessary to do a social media blackout and avoid triggering negative things such as watching the news when recovering? Maybe I\'m being silly and this is something that just applied to me but the 

In [ ]:
def convert_doccano_to_spacy(dataturks_JSON_FilePath):
    try:
        training_data = []
        lines=[]
        with open(dataturks_JSON_FilePath, 'r') as f:
            lines = f.readlines()

        for line in lines:
            data = json.loads(line)
            text = data['data']
            entities = []
            for annotation in data['label']:
                #only a single point in text annotation.
                # point = annotation['points'][0]
                # labels = annotation[2]
                # handle both list of labels or a single label.
                '''if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    #dataturks indices are both inclusive [start, end] but spacy is not [start, end)
                    entities.append((point['start'], point['end'] + 1 ,label))'''

                entities.append( ( annotation[0], annotation[1], annotation[2] ) )


            training_data.append((text, {"entities" : entities}))

        return training_data
    except Exception as e:
        logging.exception("Unable to process " + dataturks_JSON_FilePath + "\n" + "error = " + str(e))
        return None

In [ ]:
import string

def char_span_reverse( index, doc ):
  i = 0
  final_index = 0
  for token in doc:
    if( i >= index ):
      break
    final_index += ( len( token ) + ( str( token ) not in string.punctuation and "\u2019" not in str( token ) ) )
    i += 1
  return final_index

In [ ]:
a = 3
b = ["I", "like", "eating", "dumplings"]
c = "I like eating dumplings"

a = 6
b = ["I", ",", "Mike", ",", "like", "eating", "dumplings"]
char_span_reverse( a, b )

21

In [ ]:
stringy = "dfg\n\nsdf"

stringy[5]

's'

In [ ]:
def train_spacy(train_path, test_path):

    TRAIN_DATA = convert_dataturks_to_spacy(train_path)
    nlp = spacy.blank('en')  # create blank Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
       

    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(10):
            print("Starting iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)

    examples = convert_doccano_to_spacy(test_path)
    tp=0
    tr=0
    tf=0

    ta=0
    c=0        
    for text,annot in examples:

        print( "GREETINGS, WE ARE PLAYING WITH DOCUMENT NUMBER", c )
        f=open("resume"+str(c)+".txt","w")
        f1 = open( "test" + str(c) + ".json", "w" )
        doc_to_test=nlp(text)
        # print( "Ok dudes this is what doc_to_test looks like:\n", doc_to_test )
        d={}
        for ent in doc_to_test.ents:
            d[ent.label_]=[]
        for ent in doc_to_test.ents:
            d[ent.label_].append(ent.text)

        data = \
        {
            "content": text,
            "annotation": []
        }

        '''for entity in annot['entities']:
            dict_new = {
                "label": entity[2],
                "points": [{
                    "start": entity[0],
                    "end": entity[1] - 1,
                    "text": text[entity[0]:entity[1]]
                }]
            } 
            data['annotation'].append( dict_new )
        json_object = json.dumps( data )
        f1.write( json_object )'''

        for i in set(d.keys()):

            f.write("\n\n")
            f.write(i +":"+"\n")
            for j in set(d[i]):
                f.write(j.replace('\n','')+"\n")


            # print( "we iterating and the i is: ", i )
        d={}
        for ent in doc_to_test.ents:
            d[ent.label_]=[0,0,0,0,0,0]
            # print( "we seem to have initialized the zeroes" )
        for ent in doc_to_test.ents:
            # print( "doing each entity right meow and ent is", ent, "and label is", ent.label_ )
            doc_gold_text= nlp.make_doc(text)
            gold = GoldParse(doc_gold_text, entities=annot.get("entities"))
            y_true = [ent.label_ if ent.label_ in x else 'Not '+ent.label_ for x in gold.ner]
            y_pred = [x.ent_type_ if x.ent_type_ ==ent.label_ else 'Not '+ent.label_ for x in doc_to_test]  
            
            for index, x in enumerate( doc_to_test ):
              if( x.ent_type_ == ent.label_ ):
                dict_new = {
                    "label": ent.label_,
                    "points": [{
                        "start": char_span_reverse( index, doc_to_test ),
                        "end": char_span_reverse( index, doc_to_test ) + len( str( ent ) ) - 1,
                        "text": str( ent )
                    }]
                }
                data['annotation'].append( dict_new )
            
            if(d[ent.label_][0]==0):
                #f.write("For Entity "+ent.label_+"\n")   
                #f.write(classification_report(y_true, y_pred)+"\n")
                print( "hey there, y_true is", y_true )
                print( "also, y_pred is", y_pred )
                (p,r,f,s)= precision_recall_fscore_support(y_true,y_pred,pos_label=ent.label_,average='binary')
                a=accuracy_score(y_true,y_pred)
                d[ent.label_][0]=1
                d[ent.label_][1]+=p
                d[ent.label_][2]+=r
                d[ent.label_][3]+=f
                d[ent.label_][4]+=a
                d[ent.label_][5]+=1
                # print( "wow so big d is", d )
            # print( "hmmm so then it seems that d is now", d )
        c+=1
        # print( "oh hi, d is ", d )
        for i in d:
            print("\n For Entity "+i+"\n")
            print("Accuracy : "+str((d[i][4]/d[i][5])*100)+"%")
            print("Precision : "+str(d[i][1]/d[i][5]))
            print("Recall : "+str(d[i][2]/d[i][5]))
            print("F-score : "+str(d[i][3]/d[i][5]))
        print( data )
        render_NER( data )
        # for token in doc_to_test:
          # print( len( token ) )
        print( "AND THAT CONCLUDES DOCUMENT NUMBER", c - 1 )

In [ ]:
def find_entity( post, entity ):
  index = -1
  starts = []
  ends = []
  while( 1 ):
    index = post.find( entity, index + 1 )
    if( index < 0 ):
      break
    starts.append( index if entity[0] != ' ' else index + 1 )
    ends.append( index + len( entity ) - 1 if entity[-1] != ' ' else index + len( entity ) - 2 )
  return starts, ends

In [ ]:
def add_to_JSON( post, entity, entity_type, starts, ends, data ):
  for i in range( len( starts ) ):
    fart = starts[i]
    bend = ends[i]
    
    dict_new = {
        "label": entity_type,
        "points": [{
            "start": fart,
            "end": bend,
            "text": entity
        }]
    }

    data['annotation'].append( dict_new )



In [ ]:
boast = "I want to drink beer whenever I'm with my family. I guess it's because my family always drinks together, and certain members of my family are intense drinkers."
entity = "family"

farts, bends = find_entity( boast, entity )

print( farts )
print( bends )

for i in range( len( farts ) ):
  fart = farts[i]
  bend = bends[i]
  print( fart, bend )

[42, 74, 131]
[47, 79, 136]
42 47
74 79
131 136


In [ ]:
for i in range( len( farts ) ):
  fart = farts[i]
  bend = bends[i]

  dict_new = {
    "label": "Person",
    "points": [{
        "start": fart,
        "end": bend,
        "text": entity
    }]
  }

  data['annotation'].append( dict_new )

json_object = json.dumps( data )

with open( "/content/ejemplo.json", "a" ) as outfile:
  outfile.write( json_object )
  outfile.write( "\n" )

In [ ]:
convert_dataturks_to_spacy( "/content/ejemplo.json" )

[('glkjhdfsglkjsh',
  {'entities': [(100, 106, 'Person'), (110, 117, 'Person')]}),
 ('glkjhdfsglkjsh',
  {'entities': [(100, 106, 'Person'), (110, 117, 'Person')]}),
 ('glkjhdfsglkjsh',
  {'entities': [(100, 106, 'Person'),
    (110, 117, 'Person'),
    (42, 48, 'Person'),
    (74, 80, 'Person'),
    (131, 137, 'Person')]})]

In [ ]:
persons_list = [( "alone", ["alone", "by myself"] ), ( "one friend", ["friend ", "buddy"] ), ( "multiple friends", ["friends", "buddies"] ), ( "family", ["family", "families", "son", "daughter", "father", "mother", "mom", "dad", "brother", "sister", "bro", "uncle", "aunt"] ), ( "partner", ["partner", "husband", "wife", "hubby", "boyfriend", "girlfriend", "gf", "bf", "GF", "BF", "SO"] )]
locations_list = [( "home", ["home", "house"] ), ( "school", ["school", "university"] )]
posts = ["I ate by myself in my house yesterday, but then my family surprised me by bringing my uncle and aunt in for a house party. My bf was there as well and so was my buddy from the university. Amazing! What great friends and family I have."]
data = \
{
    "content": "",
    "annotation": []
}

In [ ]:
def get_training_data( posts ):
  for post in posts:
    data['content'] = post
    data['annotation'] = []
    for person in persons_list:
      entity, search_strings = person

      for search_string in search_strings:
        starts, ends = find_entity( post, search_string )
        add_to_JSON( post, search_string, "Person: " + entity, starts, ends, data )

    for location in locations_list:
      entity, search_strings = location

      for search_string in search_strings:
        starts, ends = find_entity( post, search_string )
        add_to_JSON( post, search_string, "Location: " + entity, starts, ends, data )
      
    json_object = json.dumps( data )

    with open( "/content/ejemplo.json", "a" ) as outfile:
      outfile.write( json_object )
      outfile.write( "\n" )

  return convert_dataturks_to_spacy( "/content/ejemplo.json" )

In [ ]:
def get_dict_index( entity ):
  return entity['start']

In [ ]:
from spacy import displacy

def render_NER( data ):
  displacy_dict = \
  {
      "text": data['content'],
      "ents": [{
          "start": data['annotation'][i]['points'][0]['start'],
          "end": data['annotation'][i]['points'][0]['end'] + 1,
          "label": data['annotation'][i]['label']
      } for i in range( len( data['annotation'] ) )]
  }

  displacy_dict['ents'].sort( key=get_dict_index )

  displacy.render( displacy_dict, manual=True, style="ent", jupyter=True )

In [ ]:
render_NER( data )

# Hedditch Data Authentication

In [ ]:
import requests
import pandas as pd

In [ ]:
# personal use script, secret token
auth = requests.auth.HTTPBasicAuth( 'JNpGyLI-oJ_drb93Dykw9g', 'KgImJaXURcpJG4Ud9-HLBsj8UnnnRQ' )

data = {'grant_type': 'password',
        'username': 'shawwquille_aww_neal',
        'password': 'rampat11' }

headers = {'User-Agent': 'SquatBotTUM/0.0.1' }

res = requests.post( 'https://www.reddit.com/api/v1/access_token', auth=auth, data=data, headers=headers )

token1 = res.json()['access_token']

headers = {**headers, **{'Authorization': f"bearer {token1}"}}

requests.get( 'https://oauth.reddit.com/api/v1/me', headers=headers )

<Response [200]>

In [ ]:
res = requests.get( "https://oauth.reddit.com/r/stopdrinking/new", headers=headers, params={'limit': '200'} )

print( res.json() )

{'kind': 'Listing', 'data': {'after': 't3_xcozly', 'dist': 100, 'modhash': None, 'geo_filter': '', 'children': [{'kind': 't3', 'data': {'approved_at_utc': None, 'subreddit': 'stopdrinking', 'selftext': 'This is a new one for me. I had to put my beautiful 14 year old dog to sleep last night. It was the hardest decision I ever made. I spent 3 days with her, fully sober, present and treating her like a queen! I was kind and gentle and, yes, I cried like a baby, but when the time came; I was quiet and calm for her, I wanted her to go peacefully, and she did. I know full well if I had been drinking I wouldve made much more of a scene and possibly distressed her further. I am so glad to be sober right now. Any grief I have dealt with in the past has been masked with alcohol. not his time. I am absolutely devastated and heart broken but will be dealing with this pain SOBER. For her.', 'author_fullname': 't2_80ounxl8', 'saved': False, 'mod_reason_title': None, 'gilded': 0, 'clicked': False, 't

In [ ]:
df = pd.DataFrame()

for post in res.json()['data']['children']:
  df = df.append({
      'subreddit': post['data']['subreddit'],
      'title': post['data']['title'],
      'selftext': post['data']['selftext'],
      'upvote_ratio': post['data']['upvote_ratio'],
      'ups': post['data']['ups'],
      'downs': post['data']['downs'],
      'score': post['data']['score']
  }, ignore_index=True )

In [ ]:
df.head()

,subreddit,title,selftext,upvote_ratio,ups,downs,score
0,stopdrinking,Grief when sober.,This is a new one for me. I had to put my beau...,1.0,1.0,0.0,1.0
1,stopdrinking,Well it finally happened,I fell asleep behind the wheel of my car a blo...,1.0,2.0,0.0,2.0
2,stopdrinking,Scared and excited to stop drinking for a whol...,Hey everyone! \n\nThis will be long and probab...,1.0,3.0,0.0,3.0
3,stopdrinking,If I don't drink today...,This will officially be the longest I've gone ...,1.0,9.0,0.0,9.0
4,stopdrinking,It's taken 9 months for the beer weight to fal...,33F and this was something I was curious about...,1.0,3.0,0.0,3.0


In [ ]:
persons_list = [( "alone", ["alone", "by myself"] ), ( "one friend", [" friend ", "buddy"] ), ( "multiple friends", [" friends", "buddies"] ), ( "family", ["family", "families", " son ", "daughter", " father", " mother", "mom", "dad", "brother", "sister", "bro ", "uncle", "aunt"] ), ( "partner", ["partner", "husband", "wife", "hubby", "boyfriend", "girlfriend", "gf", "bf", "GF", "BF", "SO"] )]
locations_list = [( "home", ["home", "house"] ), ( "school", ["school", "university"] ), ( "work", ["workplace", "job", "from work", "at work", "my work", "my office"] ), ( "bar", ["bar"] ), ( "supermarket", ["market", "store"] )]
data = \
{
    "content": "",
    "annotation": []
}

spacy_entities = get_training_data( df['selftext'] )

In [ ]:
spacy_entities

[('6 months ( Pt.3) \nGod it’s like some days I’m so proud of how good I feel. Other days, it seems impossible- the GRINDING. It’s like I’m always slightly uncomfortable. With everything. Everything is uncomfortable. The good days I must be doing something to distract the feeling, and I can’t pinpoint what it is. It’s like I can look at my life and realize how many good things sobriety has gifted me, and know the bottom, because I’ve been there. I know I can’t go back there.  Why do I want that temporary release just to go back to shit? I just want somebody maybe to tell me there is a sense of calmness eventually. It’s always an itch, for what, I don’t know. I just want it to shut the hell up. I don’t want to party, I want to take a deep breath.',
  {'entities': []}),
 ('I’ve been waiting a whole year to make this post! Definitely thought this was impossible at this time last year, but here we are. I hope this inspires someone else - if I can do it, you can do it. ❤️',
  {'entities': [

In [ ]:
train_spacy( "/content/ejemplo.json", "/content/admin.jsonl" )

/usr/local/lib/python3.7/dist-packages/spacy/language.py:639: UserWarning: [W033] Training a new parser or NER using a model with an empty lexeme normalization table. This may degrade the performance to some degree. If this is intentional or this language doesn't have a normalization table, please ignore this warning.
  **kwargs
/usr/local/lib/python3.7/dist-packages/spacy/language.py:639: UserWarning: [W034] Please install the package spacy-lookups-data in order to include the default lexeme normalization table for the language 'en'.
  **kwargs


Starting iteration 0


/usr/local/lib/python3.7/dist-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "10 weeks yesterday! I didn't think I would get thi..." with entities "[(127, 130, 'Person: family'), (310, 312, 'Person:...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/usr/local/lib/python3.7/dist-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Feeling self destructive today and I really want a..." with entities "[(195, 200, 'Location: supermarket')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/usr/local/lib/python3.7/dist-packages/spacy/language.py:482: UserWarning: [W030] Some entities could 

{'ner': 1228.160929451328}
Starting iteration 1
{'ner': 236.6474767037196}
Starting iteration 2
{'ner': 137.829671258421}
Starting iteration 3
{'ner': 93.18989651924822}
Starting iteration 4
{'ner': 46.79072670119162}
Starting iteration 5
{'ner': 36.66192192415315}
Starting iteration 6
{'ner': 90.32219952818377}
Starting iteration 7
{'ner': 13.642275931405406}
Starting iteration 8
{'ner': 19.11003880908102}
Starting iteration 9
{'ner': 20.857340247687283}
GREETINGS, WE ARE PLAYING WITH DOCUMENT NUMBER 0
{'content': '"Social media and news black out?\nI\'m at work and I just had a troubling realization. Now I\'m sure a lot of people know what\'s happened this past weekend in america. I won\'t go into it here but obviously it\'s horrific. I\'ve been feeling a little shaky all day, the urge to escape my worries and anxiety swirling in my mind.\n\nI should clarify I\'ve long been a drinker to try to combat my social anxiety and constant worry about literally anything and everything at any 

AND THAT CONCLUDES DOCUMENT NUMBER 0
GREETINGS, WE ARE PLAYING WITH DOCUMENT NUMBER 1
hey there, y_true is ['Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partner', 'Not Person: partn

...

AND THAT CONCLUDES DOCUMENT NUMBER 16
GREETINGS, WE ARE PLAYING WITH DOCUMENT NUMBER 17
{'content': '"Facing my fears\nToday is day 79 of sobriety. I don\'t have any urge to drink, but I\'ve been feeling blue the past couple of days. I\'m going to go to AA for the first time tonight. The reason I started drinking was to relieve my social anxiety.  I\'m nervous to the point of tears, and don\'t know what I\'m going to say. \n\nI suppose I\'m just posting this for accountability or validation. Wish me luck!\n\n&amp;#x200B;\n\nIWNDWYT"', 'annotation': []}


AND THAT CONCLUDES DOCUMENT NUMBER 17
GREETINGS, WE ARE PLAYING WITH DOCUMENT NUMBER 18
{'content': '"Day 6. Really want a drink...\nI\'m on day six. Really craving a drink. "', 'annotation': []}


AND THAT CONCLUDES DOCUMENT NUMBER 18
GREETINGS, WE ARE PLAYING WITH DOCUMENT NUMBER 19
{'content': '"I was 2 weeks sober then started having suicidal dreams and was waking up to suicidal thoughts. Relapsed. Those dreams and thoughts have gone away. Any advice?\nI don\'t want to keep drinking. Enough of my life has been consumed by alcohol."', 'annotation': []}


AND THAT CONCLUDES DOCUMENT NUMBER 19
GREETINGS, WE ARE PLAYING WITH DOCUMENT NUMBER 20
{'content': '"Struggling, mood low.\nMood low. Can\'t bear too look st myself in the mirror. This is a longstanding problem and lot of self hatred. The amount of self hatred and hopelessness I felt when I looked in the mirror just now has led to thoughts of drinking and thoughts of giving up I guess. On recovery but most importantly on myself. Not having a good day and days like this make me feel even more hopeless. I know these times can pass and that is what people say to try and help but for me these times or times of immense struggle are very frequent and when I feel this bad I have urges to act in the thoughts as I don\'t want to live my life when this keeps happening over and over again. I am safe, just struggling right now."', 'annotation': []}


AND THAT CONCLUDES DOCUMENT NUMBER 20


In [ ]:
with open( "/content/test14.json", "r" ) as ner_file:
  for line in ner_file.readlines():
    ner_dict = json.loads( line )
    render_NER( ner_dict )

In [ ]:
gold = [0 for i in range( 1000 )]
gold[37] = gold[316] = gold[844] = gold[987] = 1

silver = [0 for i in range( 1000 )]
silver[37] = silver[834] = 1

precision_recall_fscore_support( gold, silver, average="binary" )

(0.5, 0.25, 0.3333333333333333, None)

In [ ]:
import re 
nlp_test = spacy.blank( 'en' )
doc = nlp_test( "the first monf of the ear is yanuary" )
for match in re.finditer("monf", doc.text):
    start, end = match.span()
    print( start, end )
    span = doc.char_span(start, end)
    print( span )
    print( type(span) )
    print( type(span[0] ) )
    print( span[0].i )
    print( span[-1].i )
    
    # match = (match_id, span[0].i, span[-1].i + 1)

10 14
monf
<class 'spacy.tokens.span.Span'>
<class 'spacy.tokens.token.Token'>
2
2
